# ETL Pipeline for Pre-Processing the Files

## Part I. PRE-PROCESS FILES STORED ON S3

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/knadoor/Downloads/1 - Data Modeling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Apache Cassandra table creation, data insertion and validation

### The combined event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a cluster and a connection to the database

In [5]:
# Connect to a Cassandra instance on local machine 

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1']) 
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Connect to the Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

#### Create queries to ask the following three questions of the data:
<font color=red>
    
##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
</font>


#### Query 1: Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
# Create table structure in Cassandra
query = '''
            CREATE TABLE IF NOT EXISTS app_history (
                     artist varchar, 
                     song varchar,
                     length float,
                     sessionId int,
                     itemInSession int,
                     PRIMARY KEY (sessionId, itemInSession)
             )
        '''

try:
    session.execute(query)
except Exception as e:
    print(e)        

In [9]:
# Read each line in the data file and insert relevant fields from each row to the relevant target fields
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO app_history (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., line[0] refers to artist, line[9] refers to song,line[5] refers to length field etc.
        try:
            session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        except Exception as e:
            print(e)

In [10]:
# Validate that the data is available correctly in the table   
# Define a pandas row factory and display Apache Cassandra query output as a Pandas dataframe
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None # Needed for large queries, otherwise driver will do pagination. Default is 50000.

query = ''' 
            SELECT artist, song, length
            FROM app_history
            WHERE sessionId = 338 AND itemInSession = 4
        '''
try:
    result = session.execute(query, timeout=None)
    df = result._current_rows
except Exception as e:
    print(e)
    
df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# Create table structure in Cassandra
query = '''
            CREATE TABLE IF NOT EXISTS artist_song (
                     artist varchar, 
                     song varchar,
                     user varchar,
                     userId int,
                     sessionId int,
                     itemInSession int,
                     PRIMARY KEY (userId, sessionId, itemInSession)
             )
        '''
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Read each line in the data file and insert relevant fields from each row to the relevant target fields
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO artist_song (artist, song, user, userId, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"

        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (line[0], line[9], (line[1] + ' ' + line[4]), int(line[10]), int(line[8]), int(line[3])))
        except Exception as e:
            print(e)

In [13]:
# Validate that the data is available correctly in the table   
# Define a pandas row factory and display Apache Cassandra query output as a Pandas dataframe
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None # Needed for large queries, otherwise driver will do pagination. Default is 50000.

query = ''' 
            SELECT artist, song, user
            FROM artist_song
            WHERE userId = 10 AND sessionId = 182
        '''
try:
    result = session.execute(query, timeout=None)
    df = result._current_rows
except Exception as e:
    print(e)
    
df.head()

,artist,song,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# Create table structure in Cassandra
query = '''
            CREATE TABLE IF NOT EXISTS songs_listened (
                     user varchar, 
                     userid int,
                     song varchar,
                     PRIMARY KEY (song, userid)
             )
        '''
try:
    session.execute(query)
except Exception as e:
    print(e)          

In [15]:
# Read each line in the data file and insert relevant fields from each row to the relevant target fields
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO songs_listened (user, userid, song) "
        query = query + " VALUES (%s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, ((line[1] + ' ' + line[4]), int(line[10]), line[9]))
        except Exception as e:
            print(e)

In [16]:
# Validate that the data is available correctly in the table   
# Define a pandas row factory and display Apache Cassandra query output as a Pandas dataframe   
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None # Needed for large queries, otherwise driver will do pagination. Default is 50000.

query = ''' 
            SELECT user
            FROM songs_listened
            WHERE song = 'All Hands Against His Own'
        '''
try:
    result = session.execute(query, timeout=None)
    df = result._current_rows
except Exception as e:
    print(e)
    
df.head()

,user
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS app_history")
session.execute("DROP TABLE IF EXISTS artist_song")
session.execute("DROP TABLE IF EXISTS songs_listened");

### Close the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()